In [23]:
import torch
import pandas as pd
import numpy as np
from pathlib import Path
from typing import Dict, List
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelWithLMHead

In [24]:
!python prepare_messages.py --tg-history-path "data/result.json" --output-path "data/data.csv"

In [25]:
DATA_PATH = "data/data.csv"

data = load_dataset("csv", data_files=DATA_PATH, split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /Users/ninachely/.cache/huggingface/datasets/csv/default-3ab33023f5cffb8a/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


In [26]:
print(data)

Dataset({
    features: ['context_3', 'context_2', 'context_1', 'response'],
    num_rows: 121829
})


In [27]:
data = data.filter(lambda example: example["context_1"] != None)
print(data)

Filter:   0%|          | 0/121829 [00:00<?, ? examples/s]

Dataset({
    features: ['context_3', 'context_2', 'context_1', 'response'],
    num_rows: 79105
})


In [28]:
print(data)

Dataset({
    features: ['context_3', 'context_2', 'context_1', 'response'],
    num_rows: 79105
})


In [29]:
data[0]

{'context_3': None,
 'context_2': None,
 'context_1': 'норм цвет)',
 'response': 'Грустно видеть Диану с ником куратора\nто есть ты больше не наш куратор?(('}

In [30]:
data = data.train_test_split(test_size=0.2, shuffle=True)

In [31]:
data['train'][0:1]

{'context_3': ['регулярно чистить людей и их сообщения \nvs \nзарекомендовать себя как формо-ненавистников и никогда больше не париться насчет того, что сюда кто-то из них придет'],
 'context_2': ['эти опросы не были никогда серьезной проблемой в этом чате, их публикуют раз в никогда'],
 'context_1': ['Вряд-ли нашелся ирл чел который имеет такой же timestamp'],
 'response': ['когда социологи стали частью коллектива пми?)']}

In [36]:
FIRST_SPEAKER_TOKEN = '@@ПЕРВЫЙ@@'
SECOND_SPEAKER_TOKEN = '@@ВТОРОЙ@@'

CONTEXT_COLS = ['context_3', 'context_2', 'context_1']
RESPONSE_COL = ['response']
SEP = ' '

def convert_to_dialog(sample: Dict[str, str]) -> Dict[str, str]:
    """
        Convert sample row to dialogs str format
    """
    result_dict = dict()
    dialog = ""
    for i in range(len(CONTEXT_COLS)):
        key = CONTEXT_COLS[i]
        if key in sample and sample[key] is not None:
            speaker_token = FIRST_SPEAKER_TOKEN if i % 2 == 0 else SECOND_SPEAKER_TOKEN
            dialog += speaker_token + SEP + sample[key] + SEP
    
    response_key = RESPONSE_COL[0]
    if response_key in sample and sample[response_key] is not None:
        dialog += SECOND_SPEAKER_TOKEN + SEP + sample[response_key]

    result_dict['text'] = dialog

    return result_dict

In [37]:
tokenizer = AutoTokenizer.from_pretrained('tinkoff-ai/ruDialoGPT-medium')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [38]:
print(data['train'][0])

{'context_3': 'регулярно чистить людей и их сообщения \nvs \nзарекомендовать себя как формо-ненавистников и никогда больше не париться насчет того, что сюда кто-то из них придет', 'context_2': 'эти опросы не были никогда серьезной проблемой в этом чате, их публикуют раз в никогда', 'context_1': 'Вряд-ли нашелся ирл чел который имеет такой же timestamp', 'response': 'когда социологи стали частью коллектива пми?)'}


In [39]:
train = data['train'].map(convert_to_dialog)
test = data['test'].map(convert_to_dialog)

Map:   0%|          | 0/63284 [00:00<?, ? examples/s]

Map:   0%|          | 0/15821 [00:00<?, ? examples/s]

In [40]:
print(train[0])

{'context_3': 'регулярно чистить людей и их сообщения \nvs \nзарекомендовать себя как формо-ненавистников и никогда больше не париться насчет того, что сюда кто-то из них придет', 'context_2': 'эти опросы не были никогда серьезной проблемой в этом чате, их публикуют раз в никогда', 'context_1': 'Вряд-ли нашелся ирл чел который имеет такой же timestamp', 'response': 'когда социологи стали частью коллектива пми?)', 'text': '@@ПЕРВЫЙ@@ регулярно чистить людей и их сообщения \nvs \nзарекомендовать себя как формо-ненавистников и никогда больше не париться насчет того, что сюда кто-то из них придет @@ВТОРОЙ@@ эти опросы не были никогда серьезной проблемой в этом чате, их публикуют раз в никогда @@ПЕРВЫЙ@@ Вряд-ли нашелся ирл чел который имеет такой же timestamp @@ВТОРОЙ@@ когда социологи стали частью коллектива пми?)'}


In [41]:
def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples['text']])

In [42]:
tokenized_train = train.map(
    preprocess_function,
    remove_columns=train.column_names
)

tokenized_test_ds = test.map(
    preprocess_function,
    remove_columns=test.column_names
)

Map:   0%|          | 0/63284 [00:00<?, ? examples/s]

Map:   0%|          | 0/15821 [00:00<?, ? examples/s]

In [43]:
# https://huggingface.co/docs/transformers/tasks/language_modeling

block_size = 128


def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [44]:
lm_dataset = tokenized_train.map(group_texts, batched=True, num_proc=4)

Map (num_proc=4):   0%|          | 0/63284 [00:00<?, ? examples/s]

In [45]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [46]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained("distilgpt2")

In [ ]:
training_args = TrainingArguments(
    output_dir="my_awesome_eli5_clm-model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

trainer.train()

In [ ]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:
from huggingface_hub import notebook_login

notebook_login()